# Robust Decision Making

We need to introduce a small tolerance when comparing floating point values, to account for numerical approximations in the code. Here you can set the global value:

In [1]:
TOL = 1e-6

To solve decision problems using bounded probability, we first need some code to calculate lower and upper expectations themselves.

## Credal Sets

Here, for simplicity, we will always specify a lower expectation by means of a finite set of probability mass functions.
The lower expectation is then the minimal expectation with respect to this set.

Let's move to lower expectations. A credal set will then be a set of probability mass functions. For simplicity, we will also use a sequence to represent these in the code. For example, when coding, we will represent the credal set $\{(0.2, 0.2, 0.6), (0.4, 0.1, 0.5)\}$ as follows:

In [2]:
[[0.2, 0.2, 0.6], [0.4, 0.1, 0.5]]

[[0.2, 0.2, 0.6], [0.4, 0.1, 0.5]]

## Lower and Upper Expectations

The lower and upper expectation are simply the minimum and maximum expectation over the credal set $\mathcal{M}$:
$$\underline{E}(X)=\min_{p\in\mathcal{M}}E_p(X)\qquad\overline{E}(X)=\max_{p\in\mathcal{M}}E_p(X)$$
The next functions implement this.
We'll make use of the ``expectation`` function introduced previously.
In the code, we abstract the minimum and maximum transformations,
since we will consider an additional transformation later in the exercises,
and it makes for more compact and reusable code.

In [3]:
from collections.abc import Callable, Sequence

PMF = Sequence[float]
Gamble = Sequence[float]


def expectation(pmf: PMF, gamble: Gamble) -> float:
    return sum(p * g for p, g in zip(pmf, gamble))


def transform_expectations(
    transform: Callable[[Sequence[float]], float],  # sequence of expectations -> float
    credal_set: Sequence[PMF],
    gamble: Gamble,
) -> float:
    return transform([expectation(pmf, gamble) for pmf in credal_set])


def lower_expectation(credal_set: Sequence[PMF], gamble: Gamble) -> float:
    return transform_expectations(min, credal_set, gamble)


def upper_expectation(credal_set: Sequence[PMF], gamble: Gamble) -> float:
    return transform_expectations(max, credal_set, gamble)

Let's test this by finding the lower expectation of the gamble ``[5, 3, 1]`` with respect to the credal set ``[[0.2, 0.2, 0.6], [0.1, 0.1, 0.8]]``:

In [4]:
lower_expectation(
    credal_set=[[0.2, 0.2, 0.6], [0.1, 0.1, 0.8]],
    gamble=[5, 3, 1],
)

1.6

Let's check this value by evaluating the expectation of the gamble with respect to each member of the credal set:

In [5]:
[0.2 * 5 + 0.2 * 3 + 0.6 * 1, 0.1 * 5 + 0.1 * 3 + 0.8 * 1]

[2.2, 1.6]

The lower expectation is the lowest of these numbers, and indeed we can see that the correct value has been found.

**Exercise** Can you calculate the lower expectation of the gamble ``[1, 4, 2]`` with respect to the same credal set as the one used in the example?

In [6]:
# write your solution here

**Exercise** Can you also calculate the same gamble's upper expectation (again with the same credal set)?

In [7]:
# write your solution here

**Exercise** Verify that $\overline{E}(X)=-\underline{E}(-X)$, for the gamble ``[1, 4, 2]``.

In [8]:
# write your solution here

## Gamma-maximin

In $\Gamma$-maximin, we choose the gamble with the highest lower expectation:
$$\arg\max_{d\in D}\underline{E}(X_d)$$

In [9]:
def is_gamma_maxi_something(
    # something = gamble -> float (e.g. lower prevision, upper prevision, ...)
    something: Callable[[Gamble], float],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    values = list(map(something, gambles))
    max_value = max(values)
    return [value + TOL >= max_value for value in values]


def is_gamma_maximin(
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def something(gamble: Gamble) -> float:
        return lower_expectation(credal_set, gamble)

    return is_gamma_maxi_something(something, gambles)

Let us test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [10]:
is_gamma_maximin(
    credal_set=[[0.5, 0.5], [0.8, 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[False, False, True]

This tells us that the third gamble (and only the third one) is optimal according to $\Gamma$-maximin.

## Gamma-maximax

The situation for $\Gamma$-maximax is very similar.
We choose the gamble with the highest upper expectation:
$$\arg\max_{d\in D}\overline{E}(X_d)$$
A small change in the above ``is_gamma_maximin`` function gives us ``is_gamma_maximax``:

In [11]:
def is_gamma_maximax(
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def something(gamble: Gamble) -> float:
        # we changed just the next line
        return upper_expectation(credal_set, gamble)

    return is_gamma_maxi_something(something, gambles)

Let us again test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [12]:
is_gamma_maximax(
    credal_set=[[0.5, 0.5], [0.8, 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, False, False]

This tells us that the first gamble (and only the first one) is optimal according to $\Gamma$-maximax.

## Hurwicz

We cover this criterion as part of the exercises only. Hurwicz allows a choice inbetween $\Gamma$-maximin and $\Gamma$-maximax.
First, we fix a pessimism index $\beta\in[0,1]$.
We then choose the gamble with the highest convex combination of lower and upper expectation, as follows:
$$\arg\max_{d\in D}\beta\underline{E}(X_d)+(1-\beta)\overline{E}(X_d)$$

**Exercise** Explain why $\beta$ is called a 'pessimism' index (and not, say, an 'optimism' index).

*Write your solution here.*

We can implement the Hurwicz criterion as follows:

In [13]:
def hurwicz_expectation(
    beta: float,
    credal_set: Sequence[PMF],
    gamble: Gamble,
) -> float:
    def hurwicz(expectations: Sequence[float]) -> float:
        return beta * min(expectations) + (1 - beta) * max(expectations)

    return transform_expectations(hurwicz, credal_set, gamble)


def is_hurwicz(
    beta: float,
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def something(gamble: Gamble) -> float:
        return hurwicz_expectation(beta, credal_set, gamble)

    return is_gamma_maxi_something(something, gambles)

**Exercise** The ``hurwicz_expectation`` calculates part of the formula for the Hurwicz criterion. Which part is that?

*Write your solution here.*

**Exercise** Find the Hurwicz optimal gambles in our example with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``, for $\beta=0.5$. Confirm that the optimal choice here is ``[420, 300]`` (corresponding to neither the $\Gamma$-maximin nor the $\Gamma$-maximax solution).

In [14]:
# write your solution here

## Interval Maximality

In interval maximality, we first introduce an ordering $\sqsupset$ defined by
$$X\sqsupset Y\text{ whenever }\underline{E}(X)>\overline{E}(Y)$$
or in other words, whenever
the interval $[\underline{E}(X),\overline{E}(X)]$
completely dominates the interval $[\underline{E}(Y),\overline{E}(Y)]$.

Under interval maximality, we
choose any gamble which is undominated with respect to $\sqsupset$:
$$\{d\colon (\forall e\in D)(X_e\not\sqsupset X_d)\}$$
We can implement this as follows.
For guidance, a mathematical explanation of the variables used is as follows
(recall that $\mathcal{M}$ denotes the credal set):

* ``xs`` is $\{E_p(X)\colon p\in\mathcal{M}\}$ for some gamble $X$
* ``ys`` is $\{E_p(Y)\colon p\in\mathcal{M}\}$ for some gamble $Y$
* ``xss`` is $\{\{E_p(X_d)\colon p\in\mathcal{M}\}\colon d\in D\}$

The remaining variables should be obvious.

In [15]:
# check dominance between two vectors, using min and max values
def interval_dominates(
    xs: Sequence[float],
    ys: Sequence[float],
) -> bool:
    return min(xs) > max(ys) + TOL


def is_maximal(
    # compares two vectors
    dominates: Callable[[Sequence[float], Sequence[float]], bool],
    # sequence of vectors
    xss: Sequence[Sequence[float]],
) -> Sequence[bool]:
    def is_not_dominated(xs: Sequence[float]) -> bool:
        return all(not dominates(ys, xs) for ys in xss)

    return [is_not_dominated(xs) for xs in xss]


def is_interval_maximal(
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    xss = [[expectation(pmf, gamble) for pmf in credal_set] for gamble in gambles]
    return is_maximal(interval_dominates, xss)

**Exercise** In the function ``interval_dominates``, explain what the expressions ``min(xs)`` and ``max(xs)`` correspond to.

*Write your solution here.*

Let us test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [16]:
is_interval_maximal(
    credal_set=[[0.5, 0.5], [0.8, 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, True, True]

## Robust Bayes Maximality

We say that $X$ robust Bayes dominates $Y$, and write $X\succ Y$,
whenever
$$\forall p\in\mathcal{M}\colon E_p(X)>E_p(Y)$$
or equivalently,
$$\underline{E}(X-Y)>0$$

Under robust Bayes maximality, we
choose any gamble which is undominated with respect to $\succ$:
$$\{d\colon (\forall e\in D)(X_e\not\succ X_d)\}$$
We can implement this as follows, recycling our ``is_maximal`` function from before:

In [17]:
# check dominance between two vectors, pointwise
def pointwise_dominates(
    xs: Sequence[float],
    ys: Sequence[float],
) -> bool:
    return all(x > y + TOL for x, y in zip(xs, ys))


def is_rbayes_maximal(
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    xss = [[expectation(pmf, gamble) for pmf in credal_set] for gamble in gambles]
    return is_maximal(pointwise_dominates, xss)

**Exercise** Compare the implementation of ``is_rbayes_maximal`` with the implementation of ``is_interval_maximal``. There is just one difference. Can you identify it?

*Write your solution here.*

Let us test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [18]:
is_rbayes_maximal(
    credal_set=[[0.5, 0.5], [0.8, 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, True, True]

## Robust Bayes Admissibility

Under robust Bayes admissibility, we choose all decisions
that have maximal expectation with respect to some $p\in\mathcal{M}$:
$$\bigcup_{p\in\mathcal{M}}\arg\max_{d\in D} E_p(X_d)$$
In code, we can implement this simply as follows:

In [19]:
def is_rbayes_admissible(
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    def arg_max(pmf: PMF) -> Sequence[bool]:
        xs = [expectation(pmf, gamble) for gamble in gambles]
        max_xs = max(xs)
        return [x + TOL >= max_xs for x in xs]

    def union(bss: Sequence[Sequence[bool]]) -> Sequence[bool]:
        return [any(bs) for bs in zip(*bss)]

    return union([arg_max(pmf) for pmf in credal_set])

Let us test this on our example, with credal set ``[[0.5, 0.5], [0.8 , 0.2]]`` and gambles ``[[440, 260], [420, 300], [370, 370]]``:

In [20]:
is_rbayes_admissible(
    credal_set=[[0.5, 0.5], [0.8, 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, False, True]

We note that robust Bayes admissibility is not just determined by the extreme points of the credal set. For this reason, normally, the credal set is assumed convex, and in that case the resulting criterion is called *E-admissibility*. The following example demonstrates this:

In [21]:
is_rbayes_admissible(
    credal_set=[[0.5, 0.5], [0.65, 0.35], [0.8, 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, True, True]

**Exercise** Using the code below, or otherwise, confirm that ``[0.65, 0.35]`` belongs to the convex hull of ``[[0.5, 0.5], [0.8 , 0.2]]``.

In [22]:
def combine(
    alpha: float,
    xs: Sequence[float],
    ys: Sequence[float],
) -> Sequence[float]:
    return [(1 - alpha) * x + alpha * y for x, y in zip(xs, ys)]


alpha = 0  # you'll need to tweak this value
combine(alpha, [0.5, 0.5], [0.8, 0.2])

[0.5, 0.5]

## Additional Exercises

**Exercise** Consider again the same very simple example from the lectures:

> A company makes a product, and believes in increasing future demand.  The manager asks you, the decision expert, whether he should buy new machinery, use overtime, or do nothing. The upcoming year, demand can either increase or remain the same.
>
> If we buy new machinery, then the profit at the end of the year
will be $440$ (in thousands of pounds) if demand increases, and
$260$ otherwise. Alternatively, if we use overtime, then the
profit will be $420$ if demand increases, and $300$ otherwise. If
we do nothing, profit will be $370$.

We have done additional market research, and we now know that demand will increase with probability at least $0.6$, and at most $0.65$. What advice can we give the manager now? Investigate with each optimality criterion. Hint: The credal set $\mathcal{M}$ is now ``[[0.6, 0.4], [0.65, 0.35]]``.

In [23]:
# write your solution here

**Exercise** You have the option to invest some money. The market can either improve, remain, or worsen. Your set of probabilities are tabulated below. You have the choice between 4 investment options, summarized in the table below.

| option | improve | remain | worsen
|---|---|---|---
| #1 | 100 | 50 | -25
| #2 | 75 | 50 | 0
| #3 | 60 | 55 | 10
| #4 | 35 | 35 | 35

The credal set consists of two probability mass functions:

improve | remain | worsen
---|---|---
0.0 | 0.6 | 0.4
0.3 | 0.3 | 0.4

1. Write down the credal set and the set of gambles in code.
2. Find the optimal investment options under each decision criterion that we have covered.
3. Now, consider interval maximality and robust Bayes maximality. Which of these two criteria gives a 'better' answer, and why?

In [24]:
# write your solution here

**Exercise** Consider the credal set ``[[1, 0], [0.5, 0.5], [0, 1]]`` and the gambles ``[[10, 0], [4, 4], [0, 10]]``. Show that this decision problem has a $\Gamma$-maximin gamble that is not robust Bayes admissible.

In [25]:
# write your solution here

**Exercise** We proved in the lectures that the set of interval maximal gambles is equal to
$$\left\{d\colon \overline{E}(X_d)\ge\max_{e\in D}\underline{E}(X_e)\right\}$$
Can you implement this as a function?
You can use the code below as a starting point.

In [26]:
def is_interval_maximal_2(
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    xss = [[expectation(pmf, gamble) for pmf in credal_set] for gamble in gambles]
    maxmin = max(min(xs) for xs in xss)
    maxs = [max(xs) for xs in xss]
    return  # ... complete this line!


# test the result
is_interval_maximal_2(
    credal_set=[[0.5, 0.5], [0.8, 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

**Exercise** We saw in the lectures that we do not need to compare all gambles, because every non-maximal element is dominated by a maximal element. This holds not just for robust Bayes maximality, but for every transitive ordering. Can we improve the ``is_maximal`` function so it immediately eliminates non-maximal sequences from consideration, leading to a more efficient implementation? Recall that when writing ``a and b`` in Python, where ``a`` and ``b`` are arbitrary expressions, ``b`` is never evaluated unless ``a`` is ``True``. 

1. Explain how the code for the ``is_maximal_2`` function below achieves a more efficient implementation.
2. Explain why, when using the implementation below for ``is_rbayes_maximal_2``, it pays off to sort the gambles in advance, so that maximal gambles are likely come last in the sequence.
3. (Challenging) How could you improve ``is_rbayes_maximal_2`` function to do precisely that? Hint: What happens if you sort the gambles by expectation of, say, some element of ``credal_set``? You are not asked to actually implement this improvement; it is enough to simply comment on how you would do so.

In [27]:
def is_maximal_2(
    # compares two vectors
    dominates: Callable[[Sequence[float], Sequence[float]], bool],
    # sequence of vectors
    xss: Sequence[Sequence[float]],
) -> Sequence[bool]:
    if not xss:
        return []
    else:
        ys = xss[0]
        zss = xss[1:]
        is_max_zss = is_maximal_2(dominates, zss)
        is_ys_dominated = any(
            is_max_zs and dominates(zs, ys) for zs, is_max_zs in zip(zss, is_max_zss)
        )
        is_max_zss_2 = (
            is_max_zss
            if is_ys_dominated
            else [
                is_max_zs and not dominates(ys, zs)
                for zs, is_max_zs in zip(zss, is_max_zss)
            ]
        )
        return [not is_ys_dominated] + is_max_zss_2


def is_rbayes_maximal_2(
    credal_set: Sequence[PMF],
    gambles: Sequence[Gamble],
) -> Sequence[bool]:
    xss = [[expectation(pmf, gamble) for pmf in credal_set] for gamble in gambles]
    return is_maximal_2(pointwise_dominates, xss)


is_rbayes_maximal_2(
    credal_set=[[0.5, 0.5], [0.8, 0.2]],
    gambles=[[440, 260], [420, 300], [370, 370]],
)

[True, True, True]

**Exercise** Show that, for the special case
when the decision problem has only two options (i.e. only two gambles are involved),
robust Bayes maximality and robust Bayes admissibility coincide.

*Write your solution here.*

**Exercise** Consider a situation with a known fixed probability mass function for the likelihood $p(y|x)$ and a vacuous credal set for the parameter $x$.
Show that, for any function $Z$ of $x$ and $y$, we have that
$$\min_x\underbrace{\sum_y Z(x,y)p(y|x)}_{E(Z|x)}
=\min_{p\in\mathcal{M}} \underbrace{\sum_x\sum_y Z(x,y)p(x,y)}_{E_p(Z)}$$
or in other words, that
$$\min_x E(Z|x)=\underline{E}(Z)$$
Note that the expectations on the left hand sides are precise since only the likelihood is used, which is assumed fixed and known.

*Write your solution here.*

**Exercise** In this exercise we prove a version of Wald's theorem. Consider the situation from the previous exercise.

1. Show that a strategy $\delta$ is Wald admissible,
in the sense that
$$\forall\delta',\,\exists x\colon\quad E(U(\delta(Y),x)|x)\ge E(U(\delta'(Y),x)|x)$$
if and only if it is robust Bayes maximal, in the sense that
$$\forall\delta',\,\exists p\in\mathcal{M}\colon\quad E_p(U(\delta(Y),X))\ge E_p(U(\delta'(Y),X))$$
Hint: Use the equality from the result from above with $Z(x,y)=U(\delta(y),x)-U(\delta'(y),x)$.

2. (Challenging) What can you say about the relationship between robust Bayes maximality with respect to the posterior lower expectation $\underline{E}(\cdot|y)$ and Wald admissibility (in the sense as above, i.e. with respect to the likelihood)?

3. The definition of Wald admissibility as defined in the lectures
is not equivalent to the definition of Wald admissibility in this exercise.
Identify the difference.

*Write your solution here.*

**Exercise** Show that $\Gamma$-maximin, $\Gamma$-maximax, and Hurwicz (if you did the exercises on this topic) gambles are always interval maximal.

*Write your solution here.*

**Exercise** Show that every $\Gamma$-maximax gamble is robust Bayes admissible.

*Write your solution here.*

**Exercise** Show that every robust Bayes admissible gamble is also robust Bayes maximal.

*Write your solution here.*

**Exercise** Show that every robust Bayes maximal gamble is also interval maximal.

*Write your solution here.*

**Exercise** Show that every $\Gamma$-maximin gamble is robust Bayes maximal.

*Write your solution here.*